In [22]:
from model import utils
from model import datagen
import os
import pandas
import numpy
import rasterio
from rasterio.windows import Window

In [2]:
train_df = pandas.read_csv(utils.TRAIN_PATH).set_index('id')
train_df

,encoding
id,
aaa6a05cc,30989109 59 31007591 64 31026074 68 31044556 7...
b2dc8411c,56157731 21 56172571 45 56187411 51 56202252 5...
54f2eec69,124967057 36 124997425 109 125027828 147 12505...
e79de561c,7334642 14 7350821 41 7367001 67 7383180 82 73...
2f6ecfcdf,296084587 4 296115835 6 296115859 14 296147109...
0486052bb,101676003 6 101701785 8 101727568 9 101753351 ...
1e2425f28,49453112 7 49479881 22 49506657 31 49533433 40...
afa5e8098,65837968 7 65874765 11 65874827 12 65911562 15...
095bf7a1f,113277795 21 113315936 53 113354083 87 1133922...


In [14]:
tile_size = utils.GLOBALS['tile_size']
min_overlap = utils.GLOBALS['min_overlap']
tile_size, min_overlap

(256, 32)

In [29]:
slices_df = pandas.DataFrame(columns=['id', 'x1', 'x2', 'y1', 'y2', 'glom'])

for name, encoding in train_df.iterrows():
    rle = encoding.values[0]
    path = os.path.join(utils.TRAIN_DIR, f'{name}.tiff')
    raster = rasterio.open(path, transform=rasterio.Affine(1, 0, 0, 0, 1, 0))
    layers = None if raster.count == 3 else [rasterio.open(sub_dataset) for sub_dataset in raster.subdatasets]
    slices = datagen._make_grid(raster.shape, tile_size, min_overlap)
    full_mask = datagen.encoding_to_mask(
        encoding=rle,
        shape=(raster.shape[1], raster.shape[0]),
    )
    for x1, x2, y1, y2 in slices:
        window = Window.from_slices((x1, x2), (y1, y2))
        image = numpy.zeros((tile_size, tile_size, 3), dtype=numpy.uint8)
        if raster.count == 3:
            image[:, :, :] = numpy.moveaxis(raster.read([1, 2, 3], window=window), 0, -1)
        else:
            for channel in range(3):
                image[:, :, channel] = layers[channel].read(window=window)
        
        if datagen._filter_tissue(image):
            continue
        
        mask = numpy.zeros((tile_size, tile_size), dtype=numpy.uint8)
        mask[:, :] = full_mask[x1:x2, y1:y2]
        
        row = {
            'id': name,
            'x1': x1,
            'x2': x2,
            'y1': y1,
            'y2': y2,
            'glom': int(mask.sum() > 0),
        }
        slices_df = slices_df.append(row, ignore_index=True)
    break

print(slices_df.shape)
slices_df.head()

/home/nishaq/Documents/kaggle/.kaggle/lib/python3.8/site-packages/rasterio/__init__.py:207: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


(2418, 6)


,id,x1,x2,y1,y2,glom
0,aaa6a05cc,0,256,5955,6211,0
1,aaa6a05cc,0,256,6175,6431,0
2,aaa6a05cc,222,478,4852,5108,0
3,aaa6a05cc,222,478,5955,6211,0
4,aaa6a05cc,222,478,6175,6431,0


In [30]:
sum(slices_df['glom'] == 1)

426

In [31]:
sum(slices_df['glom'] != 1)

1992